In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2017
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
response

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'q': '2017',
  'records_format': 'objects',
  'resource_id': 'd73ee828-c2c6-485c-91dc-c26bd9ce3991',
  'total_estimation_threshold': None,
  'records': [{'_id': 2257,
    'OBJECTID': '2257',
    'Year': '2017',
    'State': 'CA',
    'Agency': 'CDF',
    'Unit ID': 'TCU',
    'Fire Name': 'POINT',
    'Local Incident Number': '00012170',
    'Alarm Date': '10/9/2017 12:00:00 AM',
    'Containment Date': '10/9/2017 12:00:00 AM',
    'Cause': '11',
    'Collection Method': '1',
    'Management Objective': '1',
    'GIS Calculated Acres': '130.2357',
    'Comments': 'October 2017 Siege',
    'Complex Name': None,
    'IRWIN ID': '{DAF778E8-6421-473B-A17E-CCFC0172B16E}',
    'Fire Number (historical use)': None,
    'Complex ID': None,
    'DECADES': '2010',
    'Shape__Area': '859542.81640625',
    'Shape__Length': '7792.198462983',
    'rank

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2017 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2017.columns if col not in columns_to_keep]
df_2017 = df_2017.drop(columns=columns_to_drop)
df_2017 = df_2017.dropna (subset = ["Containment Date"])
df_2017.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2017,10/9/2017 12:00:00 AM,10/9/2017 12:00:00 AM,11,130.2357
1,2017,9/11/2017 12:00:00 AM,9/11/2017 12:00:00 AM,1,59.3685
3,2017,9/19/2017 12:00:00 AM,9/19/2017 12:00:00 AM,11,38.80263
4,2017,8/31/2017 12:00:00 AM,8/31/2017 12:00:00 AM,14,0.02271913
5,2017,5/23/2017 12:00:00 AM,5/24/2017 12:00:00 AM,14,10.04382


In [4]:
df_2017['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2017['Alarm Date']]
df_2017['Containment Date'] = [str(item) for item in df_2017['Containment Date']]
df_2017['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2017['Containment Date']]
df_2017.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2017,10/9/2017,10/9/2017,11,130.2357
1,2017,9/11/2017,9/11/2017,1,59.3685
3,2017,9/19/2017,9/19/2017,11,38.80263
4,2017,8/31/2017,8/31/2017,14,0.02271913
5,2017,5/23/2017,5/24/2017,14,10.04382


In [5]:
df_2017 = df_2017.rename(columns={'Cause': 'Cause #'})
df_2017['Cause'] = ''
df_2017['Cause #'] = df_2017['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2017['Cause'] = df_2017['Cause #'].apply(map_cause_to_meaning)
df_2017.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2017,10/9/2017,10/9/2017,11,130.2357,Powerline
1,2017,9/11/2017,9/11/2017,1,59.3685,Lightning
3,2017,9/19/2017,9/19/2017,11,38.80263,Powerline
4,2017,8/31/2017,8/31/2017,14,0.02271913,Unknown / Unidentified
5,2017,5/23/2017,5/24/2017,14,10.04382,Unknown / Unidentified


In [6]:
df_2017.to_csv("outputs/fires_2017.csv",
                  encoding="utf-8", index=False, header=True)

In [7]:
# Attempt to convert 'Alarm Date' to datetime, coercing errors
df_2017['Alarm Date'] = pd.to_datetime(df_2017['Alarm Date'], errors='coerce')

# Display rows with invalid dates
invalid_dates = df_2017[df_2017['Alarm Date'].isna()]
print(invalid_dates)

# Continue with the processing
df_byalarm = df_2017.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

Empty DataFrame
Columns: [Year, Alarm Date, Containment Date, Cause #, GIS Calculated Acres, Cause]
Index: []


,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,132,2017,2017-01-30,1/30/2017,14,0.003896726,Unknown / Unidentified
1,68,2017,2017-02-14,2/14/2017,5,16.0521,Debris
2,521,2017,2017-03-08,4/8/2017,11,33.345,Powerline
3,522,2017,2017-03-08,8/8/2017,10,53.36273,Vehicle
4,331,2017,2017-03-26,4/1/2017,14,66.99863,Unknown / Unidentified


In [8]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2017-01-30 00:00:00')

In [9]:
# Convert 'Containment Date' to datetime, coercing errors
df_2017['Containment Date'] = pd.to_datetime(df_2017['Containment Date'], errors='coerce')

# Drop rows with invalid 'Containment Date'
df_2017_filtered = df_2017.dropna(subset=['Containment Date'])

# Now you can sort and process the DataFrame as needed
df_bycontainment = df_2017_filtered.sort_values(by='Containment Date', ascending=False).reset_index(drop=True)
latest_containment = df_bycontainment['Containment Date'].iloc[0]

df_bycontainment.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2017,2017-08-20,2018-01-15,1,48.382,Lightning
1,2017,2017-08-12,2018-01-15,1,31.87081,Lightning
2,2017,2017-12-04,2018-01-12,9,281790.9,Miscellaneous
3,2017,2017-08-10,2018-01-09,1,32893.26,Lightning
4,2017,2017-12-07,2017-12-24,14,4080.615,Unknown / Unidentified


In [10]:
latest_containment

Timestamp('2018-01-15 00:00:00')

In [11]:
output_path = os.path.join("outputs", "timeline_dates_2017.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])